# VERSIÓN 1

In [ ]:
from pyspark.sql import SQLContext
from pyspark import SparkContext

from pyspark.sql.types import *

from pyspark.sql.functions import col

sc = SparkContext()
sqlContext = SQLContext(sc)

# Definición del esquema para los datos
customSchema = StructType([
    StructField("clean_text", StringType()), 
    StructField("category", StringType())])

# Nombres de los archivos CSV que contienen los datos de tweets
filename1 = './Twitter_Data.csv'
filename2 = './Twitter_Data.csv'

# Lectura de datos desde los archivos CSV
df1 = sqlContext.read.format("csv").option("header", "true").schema(customSchema).load(filename1)
df2 = sqlContext.read.format("csv").option("header", "true").schema(customSchema).load(filename2)

# Unión de los datos de los dos archivos
df = df1.union(df2)

# Eliminación de filas con valores nulos
data = df.na.drop(how='any')

# Agrupación de los datos por la columna "category" y conteo de las categorías
data.groupBy("category").count().orderBy(col("count").desc())

# Configuración de transformaciones para procesar el texto
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression

# Tokenización del texto utilizando una expresión regular
regexTokenizer = RegexTokenizer(inputCol="clean_text", outputCol="words", pattern="\\W")

# Definición de palabras vacías (stop words)
add_stopwords = ["http","https","amp","rt","t","c","the"]
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)

# Creación de una representación de "bag of words" a partir de las palabras tokenizadas
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=30000, minDF=5)

# Configuración de una canalización (pipeline) para aplicar las transformaciones
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

# Conversión de la columna "category" a etiquetas numéricas
label_stringIdx = StringIndexer(inputCol="category", outputCol="label")

# Definición de la canalización
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])

# Ajuste de la canalización a los datos
pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)

# División de los datos en conjuntos de entrenamiento y prueba
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed=100)

# Entrenamiento de un modelo de regresión logística
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)

# Realización de predicciones en el conjunto de prueba
predictions = lrModel.transform(testData)

# Mostrar las 10 primeras predicciones ordenadas por probabilidad
predictions.filter(predictions['prediction'] == 0).select("clean_text", "category", "probability", "label", "prediction")\
    .orderBy("probability", ascending=False).show(n=10, truncate=30)


# VERSIÓN 2

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import col

# Crear una instancia de SparkSession
spark = SparkSession.builder.appName("NombreDeLaApp").getOrCreate()

# Definición del esquema para los datos
customSchema = StructType([
    StructField("clean_text", StringType()), 
    StructField("category", StringType())])

# Nombres de los archivos CSV que contienen los datos de tweets
filename1 = './Twitter_Data.csv'
filename2 = './Twitter_Data.csv'

# Lectura de datos desde los archivos CSV
df1 = spark.read.format("csv").option("header", "true").schema(customSchema).load(filename1)
df2 = spark.read.format("csv").option("header", "true").schema(customSchema).load(filename2)

# Unión de los datos de los dos archivos
df = df1.union(df2)

# Eliminación de filas con valores nulos
data = df.na.drop(how='any')

# Agrupación de los datos por la columna "category" y conteo de las categorías
data.groupBy("category").agg(F.count("*").alias("count")).orderBy(col("count").desc())

# Configuración de transformaciones para procesar el texto
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression

# Tokenización del texto utilizando una expresión regular
regexTokenizer = RegexTokenizer(inputCol="clean_text", outputCol="words", pattern="\\W")

# Definición de palabras vacías (stop words)
add_stopwords = ["http","https","amp","rt","t","c","the"]
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)

# Creación de una representación de "bag of words" a partir de las palabras tokenizadas
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=30000, minDF=5)

# Configuración de una canalización (pipeline) para aplicar las transformaciones
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

# Conversión de la columna "category" a etiquetas numéricas
label_stringIdx = StringIndexer(inputCol="category", outputCol="label")

# Definición de la canalización
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])

# Ajuste de la canalización a los datos
pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)

# División de los datos en conjuntos de entrenamiento y prueba
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed=100)

# Entrenamiento de un modelo de regresión logística
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)

# Realización de predicciones en el conjunto de prueba
predictions = lrModel.transform(testData)

# Mostrar las 10 primeras predicciones ordenadas por probabilidad
predictions.filter(predictions['prediction'] == 0).select("clean_text", "category", "probability", "label", "prediction")\
    .orderBy("probability", ascending=False).show(n=10, truncate=30)

# Detener la sesión de pyspark
spark.stop()

+------------------------------+--------+------------------------------+-----+----------+
|                    clean_text|category|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|thanks for your great thoug...|       1|[0.9999087320241196,1.97952...|  0.0|       0.0|
|agree take crisis 2008 tank...|       1|[0.9992259697306478,1.78501...|  0.0|       0.0|
|much love too the beautiful...|       1|[0.9990916118296179,2.50813...|  0.0|       0.0|
|appeasement hai mumkeen hai...|       1|[0.9989114628831617,5.05673...|  0.0|       0.0|
|absolutelyyoure talented ac...|       1|[0.9988818167662314,3.78869...|  0.0|       0.0|
|absolutelyyoure talented ac...|       1|[0.9988818167662314,3.78869...|  0.0|       0.0|
|narendra modi interview bjp...|       1|[0.9987217361806545,5.27624...|  0.0|       0.0|
|vote for pappu straight mig...|       1|[0.9986708302727559,1.03981...|  0.0|       0.0|
|vote for 

# VERSIÓN 3

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType

# Crear una instancia de SparkSession
spark = SparkSession.builder.appName("SocialApp").getOrCreate()

# Definición del esquema para los datos
customSchema = StructType([
    StructField("clean_text", StringType()), 
    StructField("category", StringType())])

# Nombres de los archivos CSV que contienen los datos de tweets
filename1 = './Twitter_Data.csv'
filename2 = './Twitter_Data.csv'

# Lectura de datos desde los archivos CSV
df1 = spark.read.format("csv").option("header", "true").schema(customSchema).load(filename1)
df2 = spark.read.format("csv").option("header", "true").schema(customSchema).load(filename2)

# Unión de los datos de los dos archivos
df = df1.union(df2)

# Eliminación de filas con valores nulos
data = df.na.drop(how='any')

# Agrupación de los datos por la columna "category" y conteo de las categorías
data.groupBy("category").count().orderBy(col("count").desc())

# Configuración de transformaciones para procesar el texto
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression

# Tokenización del texto utilizando una expresión regular
regexTokenizer = RegexTokenizer(inputCol="clean_text", outputCol="words", pattern="\\W")

# Definición de palabras vacías (stop words)
add_stopwords = ["http","https","amp","rt","t","c","the"]
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)

# Creación de una representación de "bag of words" a partir de las palabras tokenizadas
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=30000, minDF=5)

# Configuración de una canalización (pipeline) para aplicar las transformaciones
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

# Conversión de la columna "category" a etiquetas numéricas
label_stringIdx = StringIndexer(inputCol="category", outputCol="label")

# Definición de la canalización
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])

# Ajuste de la canalización a los datos
pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)

# División de los datos en conjuntos de entrenamiento y prueba
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed=100)

# Entrenamiento de un modelo de regresión logística
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)

# Realización de predicciones en el conjunto de prueba
predictions = lrModel.transform(testData)

# Mostrar las 10 primeras predicciones ordenadas por probabilidad
predictions.filter(predictions['prediction'] == 0).select("clean_text", "category", "probability", "label", "prediction")\
    .orderBy("probability", ascending=False).show(n=10, truncate=30)

# Detener la sesión de pyspark
spark.stop()

+------------------------------+--------+------------------------------+-----+----------+
|                    clean_text|category|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|thanks for your great thoug...|       1|[0.9999087320241196,1.97952...|  0.0|       0.0|
|agree take crisis 2008 tank...|       1|[0.9992259697306474,1.78501...|  0.0|       0.0|
|much love too the beautiful...|       1|[0.9990916118296177,2.50813...|  0.0|       0.0|
|appeasement hai mumkeen hai...|       1|[0.9989114628831619,5.05673...|  0.0|       0.0|
|absolutelyyoure talented ac...|       1|[0.998881816766231,3.788692...|  0.0|       0.0|
|absolutelyyoure talented ac...|       1|[0.998881816766231,3.788692...|  0.0|       0.0|
|narendra modi interview bjp...|       1|[0.9987217361806543,5.27624...|  0.0|       0.0|
|vote for pappu straight mig...|       1|[0.9986708302727562,1.03981...|  0.0|       0.0|
|vote for 

# VERSIÓN 4

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType

# Crear una instancia de SparkSession
spark = SparkSession.builder.appName("SocialApp").getOrCreate()

# Definición del esquema para los datos
customSchema = StructType([
    StructField("clean_text", StringType()), 
    StructField("category", StringType())])

# Nombres de los archivos CSV que contienen los datos de tweets
filename = './Twitter_Data.csv'

# Lectura de datos desde los archivos CSV
df = spark.read.format("csv").option("header", "true").schema(customSchema).load(filename)

# Eliminación de filas con valores nulos
data = df.na.drop(how='any')

# Agrupación de los datos por la columna "category" y conteo de las categorías
data.groupBy("category").count().orderBy(col("count").desc())

# Configuración de transformaciones para procesar el texto
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression

# Tokenización del texto utilizando una expresión regular
regexTokenizer = RegexTokenizer(inputCol="clean_text", outputCol="words", pattern="\\W")

# Definición de palabras vacías (stop words)
add_stopwords = ["http","https","amp","rt","t","c","the"]
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)

# Creación de una representación de "bag of words" a partir de las palabras tokenizadas
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=30000, minDF=5)

# Configuración de una canalización (pipeline) para aplicar las transformaciones
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

# Conversión de la columna "category" a etiquetas numéricas
label_stringIdx = StringIndexer(inputCol="category", outputCol="label")

# Definición de la canalización
pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])

# Ajuste de la canalización a los datos
pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)

# División de los datos en conjuntos de entrenamiento y prueba
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed=100)

# Entrenamiento de un modelo de regresión logística
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)

# Realización de predicciones en el conjunto de prueba
predictions = lrModel.transform(testData)

# Mostrar las 10 primeras predicciones ordenadas por probabilidad
predictions.filter(predictions['prediction'] == 0).select("clean_text", "category", "probability", "label", "prediction")\
    .orderBy("probability", ascending=False).show(n=10, truncate=30)

# Detener la sesión de pyspark
spark.stop()

+------------------------------+--------+------------------------------+-----+----------+
|                    clean_text|category|                   probability|label|prediction|
+------------------------------+--------+------------------------------+-----+----------+
|vote for pappu straight mig...|       1|[0.9992425267693164,5.37945...|  0.0|       0.0|
|appeasement hai mumkeen hai...|       1|[0.9988912543452397,3.85556...|  0.0|       0.0|
|its always great blessing f...|       1|[0.9986654466344741,1.93836...|  0.0|       0.0|
|respected sir sar and madam...|       1|[0.9985015645235877,9.19043...|  0.0|       0.0|
|absolutelyyoure talented ac...|       1|[0.9983340032234882,4.59319...|  0.0|       0.0|
|respected sir sar and madam...|       1|[0.9983128327480089,9.81409...|  0.0|       0.0|
|respected shree narendrabha...|       1|[0.9980153739121862,0.00101...|  0.0|       0.0|
|leaders and their promises ...|       1|[0.9976862987065441,8.01212...|  0.0|       0.0|
|dear dad 